In [1]:
import pandas as pd
import numpy as np

In [2]:
import cvxpy as cp

In [16]:
df = pd.read_csv('Data/KBO_proj_20200510.csv')
df.head(12)

,Name,Position,Team,Salary,FD,DK
0,Hyun jong Yang,P,KIA,9500,30.62,18.60
1,Jake Brigham,P,KIW,8500,29.75,16.52
2,Tae hoon Kim,P,SK,6800,25.94,15.29
3,Casey Kelly,P,LG,8000,28.49,13.96
4,Yong chan Lee,P,DOO,8700,27.20,11.99
5,Dan Straily,P,LOT,6700,23.27,10.26
6,Warwick Saupold,P,HAN,9100,22.38,9.90
7,Young gyu Kim,P,NCD,4000,19.10,7.48
8,Min Kim,P,KTW,6800,18.64,6.60
9,Jung hyun Baek,P,SAM,6500,18.69,6.18


In [17]:
#read in salary
url = 'https://www.draftkings.com/lineup/getavailableplayerscsv?contestTypeId=28&draftGroupId=35317'
df_sal = pd.read_csv(url)
df_sal.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,SP,Hyun Jong Yang (14527548),Hyun Jong Yang,14527548,P,9500,KIA@SAM 05/10/2020 01:00AM ET,KIA,-2.85
1,SP,Jae Hak Lee (14527898),Jae Hak Lee,14527898,P,9500,LG@NCD 05/10/2020 01:00AM ET,NCD,15.20
2,SP,Drew Rucinski (14527549),Drew Rucinski,14527549,P,9300,LG@NCD 05/10/2020 01:00AM ET,NCD,25.30
3,SP,Warwick Saupold (14527550),Warwick Saupold,14527550,P,9100,HAN@KIW 05/10/2020 01:00AM ET,HAN,31.45
4,SP,Chris Flexen (14527899),Chris Flexen,14527899,P,9100,KTW@DOO 05/10/2020 01:00AM ET,DOO,18.70


In [18]:
#Replace names in df with names that appear in df_sal
from difflib import get_close_matches
#df['Name'] = ""
for i in np.arange(len(df)):
    curr_name = df['Name'].iloc[i]
    try:
        new_name = get_close_matches(curr_name,df_sal['Name'],cutoff=.8)[0]
        if new_name != curr_name:
            print("{} replaced with {}".format(curr_name,new_name))
    except:
        new_name = 'No Name'
        print('NO MATCH: curr_name = {}, new_name = {}'.format(curr_name,new_name))
    df['Name'].iloc[i] = new_name
#df2['Name'] = df2['Name'].apply(lambda x: get_close_matches(x,df2_sal['Name']))
df.head(5)

Hyun jong Yang replaced with Hyun Jong Yang


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Tae hoon Kim replaced with Tae Hoon Kim
Yong chan Lee replaced with Yong Chan Lee
Young gyu Kim replaced with Young Gyu Kim
Jung hyun Baek replaced with Jung Hyun Baek


,Name,Position,Team,Salary,FD,DK
0,Hyun Jong Yang,P,KIA,9500,30.62,18.60
1,Jake Brigham,P,KIW,8500,29.75,16.52
2,Tae Hoon Kim,P,SK,6800,25.94,15.29
3,Casey Kelly,P,LG,8000,28.49,13.96
4,Yong Chan Lee,P,DOO,8700,27.20,11.99


In [19]:
#Add Name + ID
df = pd.merge(df,df_sal[['Name','Name + ID']])
df.head(2)

,Name,Position,Team,Salary,FD,DK,Name + ID
0,Hyun Jong Yang,P,KIA,9500,30.62,18.60,Hyun Jong Yang (14527548)
1,Jake Brigham,P,KIW,8500,29.75,16.52,Jake Brigham (14527551)


In [21]:
df = df.dropna()

In [22]:
#Add columns for each position
positions = ['P','C','1B','2B','3B','SS','OF']
for pos in positions:
    df[pos] = np.where(df.Position.str.contains(pos),1,0)
df.head(12)

,Name,Position,Team,Salary,FD,DK,Name + ID,P,C,1B,2B,3B,SS,OF
0,Hyun Jong Yang,P,KIA,9500,30.62,18.60,Hyun Jong Yang (14527548),1,0,0,0,0,0,0
1,Jake Brigham,P,KIW,8500,29.75,16.52,Jake Brigham (14527551),1,0,0,0,0,0,0
2,Tae Hoon Kim,P,SK,6800,25.94,15.29,Tae Hoon Kim (14527928),1,0,0,0,0,0,0
3,Casey Kelly,P,LG,8000,28.49,13.96,Casey Kelly (14527910),1,0,0,0,0,0,0
4,Yong Chan Lee,P,DOO,8700,27.20,11.99,Yong Chan Lee (14527902),1,0,0,0,0,0,0
5,Dan Straily,P,LOT,6700,23.27,10.26,Dan Straily (14527556),1,0,0,0,0,0,0
6,Warwick Saupold,P,HAN,9100,22.38,9.90,Warwick Saupold (14527550),1,0,0,0,0,0,0
7,Young Gyu Kim,P,NCD,4000,19.10,7.48,Young Gyu Kim (14528001),1,0,0,0,0,0,0
8,Min Kim,P,KTW,6800,18.64,6.60,Min Kim (14527927),1,0,0,0,0,0,0
9,Jung Hyun Baek,P,SAM,6500,18.69,6.18,Jung Hyun Baek (14527557),1,0,0,0,0,0,0


In [23]:
#Top n teams
#df = df_all.copy()
df_now = df.copy()


numteams = 3
opt_teams = []
for i in np.arange(numteams): #number of teams
    n = len(df_now)
    x = cp.Variable(n,boolean=True)
    s = df_now.Salary.values
    pts = df_now.DK.values
    P = df_now['P'].values
    C = df_now['C'].values
    B1 = df_now['1B'].values
    B2 = df_now['2B'].values
    B3 = df_now['3B'].values
    SS = df_now['SS'].values
    OF = df_now['OF'].values
    constraints = [cp.sum(x) == 10,
                 x*s <= 50000,
                  x*P==2,
                  x*C==1,
                  x*B1==1,
                  x*B2==1,
                  x*B3==1,
                  x*SS==1,
                  x*OF==3]

    # Form objective.
    obj = cp.Maximize(x*pts)

    # Form and solve problem.
    prob = cp.Problem(obj, constraints)
    prob.solve()
    opt_team = df_now['Name'][x.value>.5].tolist()
    opt_team_positions = df_now['Position'][x.value>.5].tolist()
    sal = df_now[df_now.Name.isin(opt_team)].Salary.sum()
    print('Team {} (obj = {:.2f}, sal = {}): '.format(i+1,prob.value,sal))
    print(', '.join(['{} ({})'.format(player,pos) for player,pos in zip(opt_team,opt_team_positions)]))
    opt_teams.append(opt_team)
    #Remove team and go again
    df_now = df_now[~ df_now.Name.isin(opt_team)]

Team 1 (obj = 109.94, sal = 49700): 
Hyun Jong Yang (P), Jake Brigham (P), Ha Seong Kim (SS), Jae Il Oh (1B), Kun Woo Park (OF), Byung Hun Min (OF), Dong Min Han (OF), Eui Ji Yang (C), Hoon Jung (2B), Yeong Seok Jang (3B)
Team 2 (obj = 100.52, sal = 47600): 
Tae Hoon Kim (P), Casey Kelly (P), Sun Bin Kim (SS), ByungHo Park (1B), Jung Hoo Lee (OF), Preston Tucker (OF), Jun Woo Jeon (OF), Min Woo Park (2B), Yong Hwan Baek (C), Min Sung Kim (3B)
Team 3 (obj = 87.25, sal = 44700): 
Yong Chan Lee (P), Dan Straily (P), Aaron Altherr (OF), Jin Gi Jeong (OF), Chun Woong Lee (OF), Roberto Ramos (1B), Keun Woo Jeong (2B), Dong Won Park (C), Jae Ho Kim (SS), Won Seok Lee (3B)


In [27]:
#Just keep top n teams
num_teams_deep = 2
df_opt = df[df.Name.isin(list(np.hstack(opt_teams[:num_teams_deep])))]
df_opt

,Name,Position,Team,Salary,FD,DK,Name + ID,P,C,1B,2B,3B,SS,OF
0,Hyun Jong Yang,P,KIA,9500,30.62,18.60,Hyun Jong Yang (14527548),1,0,0,0,0,0,0
1,Jake Brigham,P,KIW,8500,29.75,16.52,Jake Brigham (14527551),1,0,0,0,0,0,0
2,Tae Hoon Kim,P,SK,6800,25.94,15.29,Tae Hoon Kim (14527928),1,0,0,0,0,0,0
3,Casey Kelly,P,LG,8000,28.49,13.96,Casey Kelly (14527910),1,0,0,0,0,0,0
10,Ha Seong Kim,SS,KIW,4900,14.09,10.62,Ha Seong Kim (14527576),0,0,0,0,0,1,0
11,Jae Il Oh,1B,DOO,3800,14.17,10.57,Jae Il Oh (14527604),0,0,1,0,0,0,0
12,Kun Woo Park,OF,DOO,4200,13.60,10.43,Kun Woo Park (14527593),0,0,0,0,0,0,1
14,Sun Bin Kim,SS,KIA,3400,13.66,10.06,Sun Bin Kim (14527625),0,0,0,0,0,1,0
16,ByungHo Park,1B,KIW,5000,13.33,9.77,ByungHo Park (14527572),0,0,1,0,0,0,0
18,Jung Hoo Lee,OF,KIW,4600,12.11,9.42,Jung Hoo Lee (14527580),0,0,0,0,0,0,1


In [9]:
#generate teams
### Assign exposures for top j teams
# num_teams_deep = 2
# own_gap = 5
# exposures = list(np.arange(50,50-own_gap*num_teams_deep,-own_gap))
# if len(exposures)<len(opt_teams):
#     exposures = exposures + [0]*(len(opt_teams)-len(exposures))
# exposures

[50, 45, 0]

In [15]:
df_lineups = pd.DataFrame(columns=['P','P','C','1B','2B','3B','SS','OF','OF','OF'])
df_lineups

,P,P,C,1B,2B,3B,SS,OF,OF,OF


In [33]:
#pick 2 pitchers
df_now = df_opt.copy()
list(df_now[df_now.Position=='P']['Name + ID'].sample(n=2))

['Tae Hoon Kim (14527928)', 'Jake Brigham (14527551)']

In [42]:
num_teams = 17
df_lineups = pd.DataFrame(columns=['P','P','C','1B','2B','3B','SS','OF','OF','OF'])
m = 0 #Completed teams so far
while (m < num_teams):
    #Pick pitcher
    df_now = df_opt.copy()
    #pick 2 pitchers
    curr_team = list(df_now[df_now.Position=='P']['Name + ID'].sample(n=2))
    curr_team += list(df_now[df_now.Position=='C']['Name + ID'].sample(n=1))
    curr_team  += list(df_now[df_now.Position=='1B']['Name + ID'].sample(n=1))
    curr_team += list(df_now[df_now.Position=='2B']['Name + ID'].sample(n=1))
    curr_team += list(df_now[df_now.Position=='3B']['Name + ID'].sample(n=1))
    curr_team += list(df_now[df_now.Position=='SS']['Name + ID'].sample(n=1))
    curr_team += list(df_now[df_now.Position=='OF']['Name + ID'].sample(n=3))
    
    #Check if salary is good
    sal = df_now[df_now['Name + ID'.isin(curr_team)].Salary.sum()
    print(sal)
    if sal > 50000:
        continue
    
    #Check if team has shown up before in df_lineups
    if len(df_lineups) < 1:
        old_team = False
    else:
        df3 = (df_lineups == np.array(curr_team))
        old_team = df3.transpose().all().any() #True if team is already in df_lineups
    if old_team:
        #print('Team already exists')
        continue
        
    #OK, so now we have a new team!  Add it to df_lineups
    df_lineups.loc[m] = curr_team
    m += 1
    #print(m)
    if m%10==0:
        print('m = {}'.format(m))
    
df_lineups

0
0
0
0
0
0
0
0
0
0
m = 10
0
0
0
0
0
0
0


,P,P,C,1B,2B,3B,SS,OF,OF,OF
0,Hyun Jong Yang (14527548),Jake Brigham (14527551),Eui Ji Yang (14527571),Jae Il Oh (14527604),Hoon Jung (14527682),Yeong Seok Jang (14527640),Ha Seong Kim (14527576),Jung Hoo Lee (14527580),Preston Tucker (14527573),Jun Woo Jeon (14527602)
1,Jake Brigham (14527551),Hyun Jong Yang (14527548),Yong Hwan Baek (14527641),ByungHo Park (14527572),Min Woo Park (14527591),Yeong Seok Jang (14527640),Sun Bin Kim (14527625),Jun Woo Jeon (14527602),Jung Hoo Lee (14527580),Dong Min Han (14527600)
2,Hyun Jong Yang (14527548),Tae Hoon Kim (14527928),Eui Ji Yang (14527571),ByungHo Park (14527572),Min Woo Park (14527591),Min Sung Kim (14527612),Sun Bin Kim (14527625),Dong Min Han (14527600),Kun Woo Park (14527593),Byung Hun Min (14527609)
3,Hyun Jong Yang (14527548),Tae Hoon Kim (14527928),Eui Ji Yang (14527571),ByungHo Park (14527572),Hoon Jung (14527682),Yeong Seok Jang (14527640),Ha Seong Kim (14527576),Kun Woo Park (14527593),Preston Tucker (14527573),Jung Hoo Lee (14527580)
4,Jake Brigham (14527551),Casey Kelly (14527910),Eui Ji Yang (14527571),ByungHo Park (14527572),Hoon Jung (14527682),Min Sung Kim (14527612),Sun Bin Kim (14527625),Byung Hun Min (14527609),Dong Min Han (14527600),Jun Woo Jeon (14527602)
5,Hyun Jong Yang (14527548),Casey Kelly (14527910),Yong Hwan Baek (14527641),Jae Il Oh (14527604),Min Woo Park (14527591),Min Sung Kim (14527612),Ha Seong Kim (14527576),Jun Woo Jeon (14527602),Kun Woo Park (14527593),Preston Tucker (14527573)
6,Casey Kelly (14527910),Hyun Jong Yang (14527548),Yong Hwan Baek (14527641),ByungHo Park (14527572),Hoon Jung (14527682),Yeong Seok Jang (14527640),Ha Seong Kim (14527576),Jun Woo Jeon (14527602),Byung Hun Min (14527609),Jung Hoo Lee (14527580)
7,Casey Kelly (14527910),Hyun Jong Yang (14527548),Eui Ji Yang (14527571),ByungHo Park (14527572),Hoon Jung (14527682),Yeong Seok Jang (14527640),Sun Bin Kim (14527625),Kun Woo Park (14527593),Byung Hun Min (14527609),Jung Hoo Lee (14527580)
8,Tae Hoon Kim (14527928),Hyun Jong Yang (14527548),Yong Hwan Baek (14527641),Jae Il Oh (14527604),Hoon Jung (14527682),Yeong Seok Jang (14527640),Sun Bin Kim (14527625),Kun Woo Park (14527593),Byung Hun Min (14527609),Jung Hoo Lee (14527580)
9,Tae Hoon Kim (14527928),Jake Brigham (14527551),Yong Hwan Baek (14527641),ByungHo Park (14527572),Hoon Jung (14527682),Min Sung Kim (14527612),Ha Seong Kim (14527576),Kun Woo Park (14527593),Byung Hun Min (14527609),Dong Min Han (14527600)


In [41]:
df_lineups.to_csv('Lineups/KBO_lineups.csv',index=False)